# 

# Phase 1
> 2022-2024; NA Listed Firms;

## Imports

In [1]:
import pandas as pd
import numpy as np

## Emissions Data

### Load

In [176]:
""" Read in the CSV """
raw_emissions_df = pd.read_csv(
    "phase1_emissions.csv", 
    dtype={"companyid": "str", "gvkey": "str"}, #identifiers
    parse_dates=["periodenddate"], 
)

raw_emissions_df

,institutionid,fiscalyear,periodenddate,di_319413,di_319414,di_319415,companyid,gvkey,companyname,country
0,11489,2022,2022-12-31,1859.947024,5704.373536,78880.089343,6520204,NaN,Factory Mutual Insurance Company,United States
1,11489,2022,2022-12-31,1859.947024,5704.373536,78880.089343,24951392,NaN,"Factory Mutual Insurance Company, Asset Manage...",United States
2,11489,2022,2022-12-31,1859.947024,5704.373536,78880.089343,43964734,NaN,"Allendale Mutual Insurance Co, Asset Managemen...",United States
3,11489,2023,2023-12-31,4154.828402,12742.671094,176205.682880,6520204,NaN,Factory Mutual Insurance Company,United States
4,11489,2023,2023-12-31,4154.828402,12742.671094,176205.682880,24951392,NaN,"Factory Mutual Insurance Company, Asset Manage...",United States
...,...,...,...,...,...,...,...,...,...,...
1803283,118526590,2022,2022-12-31,485.540053,1085.798816,1022.548683,111952300,NaN,Coseva Soc Coop,Italy
1803284,118706040,2022,2022-12-31,205.995561,658.395477,1709.587413,1886594015,NaN,Solà Domingo S.A.,Spain
1803285,118918440,2022,2022-07-31,40.124678,26.875831,83.556928,1887792515,NaN,"Nihonsangyo Co.,Ltd.",Japan
1803286,118991426,2022,2022-12-31,59.719416,216.809138,129.375410,1888091056,NaN,CP Management Spólka Z Ograniczona Odpowiedzia...,Poland


### EDA

**Inspect the loaded datatypes**

In [177]:
raw_emissions_df.dtypes

institutionid             int64
fiscalyear                int64
periodenddate    datetime64[ns]
di_319413               float64
di_319414               float64
di_319415               float64
companyid                object
gvkey                    object
companyname              object
country                  object
dtype: object

**The company ids are extracted and written out to a file**

In [3]:
""" Utility functions regarding validity, uniqueness, and writing out 
unique and valid ids. """

def keep_valid(data, colname=None):
    """ Only keeps the rows/items from the given dataframe/array-like which 
    have valid values - for the specified column in the case of a dataframe.

    Args:
        data: Dataframe or array-like.
        colname: Name of the dataframe column whose valid values we are using to 
            filter.

    Returns:
        The dataframe/array-like with rows/items that have invalid values, w.r.t 
        the specified column if applicable, filtered out.
    """

    # Array whose values we are interested in
    col = data[colname] if colname is not None else data
    # Values are considered valid as long as they are not NaN
    return data[pd.notnull(col)]


def extract_unique(df, colname):
    """ Extract the unique and non-NaN values from a dataframe column.
    
    Args:
        df: Dataframe.
        colname: Column name w.r.t the dataframe.

    Returns:
        Unique and non-NaN column values.
    """

    return keep_valid(pd.unique(df[colname]))


def write_ids(df, idname, filename):
    """ Writes the unique, non-NaN instances of the indicated identifier, within the 
    indicated dataframe, on separate lines of a new file, whose filename should 
    be specified.

    Args:
        df: Dataframe.
        idname: Column name of the identifier with respect to the dataframe.
        filename: The name to use for the newly created file.
    """
    with open(filename, "w") as fh:
        for idval in extract_unique(df, idname):
            fh.write(f"{idval}\n")

In [4]:
""" Export CIQ company ids """
write_ids(raw_emissions_df, "companyid", "companyids.txt")

**The number of unique company ids and gvkeys are reported for this raw emissions data**

In [105]:
""" Utility function """
def report_unique(df, colname):
    """ Reports unique, non-NaN values of a dataframe column
    
    Args:
        df: Dataframe.
        colname: Column name.
    """

    print(f"Number of unique, non-null values of \'{colname}\': {len(extract_unique(df, colname))}")

In [107]:
print("-- Raw Emissions Data")
report_unique(raw_emissions_df, "institutionid")
report_unique(raw_emissions_df, "companyid")
report_unique(raw_emissions_df, "gvkey")

-- Raw Emissions Data
Number of unique, non-null values of 'institutionid': 1720932
Number of unique, non-null values of 'companyid': 1725359
Number of unique, non-null values of 'gvkey': 24304


**Check whether every gvkey entry has a corresponding company id (non-null -> non-null)**

In [169]:
def is_backed(df, src_colname, ref_colname):
    """ Check if one column is backed by another in a dataframe. This means that 
    when the first column is non-null, the second column cannot be null as 
    otherwise it would be backing the first column.

    Args:
        df: Dataframe.
        src_colname: Name of column that should be backed.
        ref_colname: Name of backing column.
    
    Returns:
        True/False depending on whether the first column is backed by the second
    """

    # check for 0 invalid cases for valid backing
    return (df[src_colname].notnull() & df[ref_colname].isnull()).sum() == 0

def report_backed(df, src_colname, ref_colname):
    """ Reports whether every non-null value of src is backed by a non-null 
    value of ref.
    
    Args:
        df: Dataframe.
        src_colname: Name of column that should be backed.
        ref_colname: Name of backing column.
    """
    print(f"Is \'{src_colname}\' backed by \'{ref_colname}\': {'YES' if is_backed(df, src_colname, ref_colname) else 'NO'}")

In [185]:
print("-- Raw Emissions Data")
report_backed(
    raw_emissions_df, "gvkey", "companyid"
)
report_backed(
    raw_emissions_df, "companyid", "institutionid"
)

-- Raw Emissions Data
Is 'gvkey' backed by 'companyid': YES
Is 'companyid' backed by 'institutionid': YES


**Check for duplicates and null values in the 3 identifiers - companyid, gvkey and institutionid**

In [171]:
""" Utilty functions """
def report_null(df, colname):
    """ Indicate the number of nulls in the specified column in the 
    given dataframe.
    
    Args:
        df: Dataframe.
        src_colname: Name of column that we are interested in.
    """
    print(f"Number of nulls in \'{colname}\': {df[colname].isnull().sum()}")

def report_dup(df, colname):
    """ Indicate the presence of duplicates in the specified column in the 
    given dataframe.
    
    Args:
        df: Dataframe.
        src_colname: Name of column that we are interested in.
    """

    print(f"Are there duplicates in \'{colname}\': {'YES' if df[colname].duplicated().any() else 'NO'}")

In [172]:
print("-- Raw Emissions Data")
print("> Null checks")
report_null(raw_emissions_df, "companyid")
report_null(raw_emissions_df, "institutionid")
report_null(raw_emissions_df, "gvkey")
print("> Dup checks")
report_dup(raw_emissions_df, "companyid")
report_dup(raw_emissions_df, "institutionid")
report_dup(raw_emissions_df, "gvkey")

-- Raw Emissions Data
> Null checks
Number of nulls in 'companyid': 747
Number of nulls in 'institutionid': 0
Number of nulls in 'gvkey': 1769291
> Dup checks
Are there duplicates in 'companyid': YES
Are there duplicates in 'institutionid': YES
Are there duplicates in 'gvkey': YES


**Utilities for numerical value conflicts on grouping by a key. Check for one to one mapping conflicts between identifiers.**

In [184]:
""" Utility functions """

# aggregates series values, NaN if all equal, else string representation with space delimiter
# TODO: assumes no NaN in vals and also the existence of at least 1 element due 
# to the retrieval at index 0.
numer_conflict_aggr = lambda vals: np.nan if (vals.iloc[0] == vals).all() else " ".join(vals.astype(str))

def detect_numerical_conflicts(df, key, numer_cols):
    """ Detects numerical conflicts when grouping on a dataframe by the given 
    keys - filling in the multiple conflict values for inspection.
    
    Args:
        df: Dataframe.
        key: Key or keys that are used for grouping.
        numer_cols: Name of numerical columns which are checked for conflicts
    
    Returns:
        Dataframe with values as NaN if no conflicts, else the conflict values 
        delimited by spaces are filled in.
    """

    return df.groupby(key).agg(
        {
            numer_col : numer_conflict_aggr for numer_col in numer_cols
        }
    )

def one_to_one_conflict_aggr(vals):
    """ Aggregator for the 1t1 conflicts, NaN indicates valid as 1 mapped, else 
    the conflicting values of the second identifier are returned 
    (nothing or multiple).
    
    Args:
        vals: Series that is aggregated.
    Returns:
        NaN if valid, else the conflict values joined together in a string, 
        separated by spaces.
    """
    # simple short-circuit on apparent validity
    if vals.count() == 1:
        return np.nan
    # checking if there is conflict by there being no mapping
    if vals.count() == 0:
        return ""

    # otherwise, we have multiple entries (>1)
    vals = vals.dropna() # remove null bloat to leave just the entries

    if (vals.iloc[0] == vals).all(): # reduces to 1 actual mapping, hence valid
        return np.nan
    else: # otherwise, multiple actual mappings, which creates conflicts
        return " ".join(vals.astype(str))

def detect_1t1_conflicts(df, key, oth_id):
    """ Detects one to one mapping conflicts when grouping on a dataframe by the 
    first identifier and seeing how many of the second identifier this maps to.
    
    Args:
        df: Dataframe.
        key: Key for the first identifier.
        oth_id: Name of the column for the second identifier
    
    Returns:
        Dataframe with values as NaN if no conflicts, else the conflict values, 
        for the second identifier, delimited by spaces are filled in.
    """

    return df.groupby(key).agg(
        {
            oth_id : one_to_one_conflict_aggr, 
        }
    )

**Numerical value conflicts in environmental metrics when grouping by gvkey and 
fiscal year.**

In [183]:
raw_emissions_gvkey_numer_conflicts = detect_numerical_conflicts(
    raw_emissions_df, 
    ["gvkey", "fiscalyear", "periodenddate"], 
    ["di_319413", "di_319414", "di_319415"]
)

raw_emissions_gvkey_numer_conflicts

di_319413 di_319414 di_319415
gvkey  fiscalyear periodenddate                              
001004 2022       2022-05-31          NaN       NaN       NaN
001045 2022       2022-12-31          NaN       NaN       NaN
       2023       2023-12-31          NaN       NaN       NaN
001050 2022       2022-12-31          NaN       NaN       NaN
       2023       2023-12-31          NaN       NaN       NaN
...                                   ...       ...       ...
362683 2022       2022-03-31          NaN       NaN       NaN
362705 2022       2022-12-31          NaN       NaN       NaN
362758 2022       2022-03-31          NaN       NaN       NaN
362761 2022       2022-03-31          NaN       NaN       NaN
       2023       2023-03-31          NaN       NaN       NaN

[33896 rows x 3 columns]

In [187]:
raw_emissions_gvkey_numer_conflicts[
    pd.notnull(raw_emissions_gvkey_numer_conflicts["di_319413"])
    | pd.notnull(raw_emissions_gvkey_numer_conflicts["di_319414"])
    | pd.notnull(raw_emissions_gvkey_numer_conflicts["di_319415"])
]

di_319413  \
gvkey  fiscalyear periodenddate                              
003413 2022       2022-12-31        2867127.0 51062759.241   
003897 2022       2022-12-31     27114550.031 25358937.262   
004094 2022       2022-12-31      810.06 1410.524 7995.579   
005180 2022       2022-12-31        636360.11 53101.342601   
005600 2022       2022-12-31      1080082.1763 4073314.057   
...                                                    ...   
275535 2022       2022-03-31          734.626 112.41037212   
       2023       2023-03-31          611.138 80.405872624   
289724 2022       2022-12-31          101.653 0.0053786782   
295786 2022       2022-12-31       134627.057 0.9207597207   
318474 2022       2022-12-31            916.0 626.35933659   

                                               di_319414  \
gvkey  fiscalyear periodenddate                            
003413 2022       2022-12-31      1018275.0 3933.0872579   
003897 2022       2022-12-31       197000.0 1953.2613298   
004094 2022       2022-12-31     7696.0 2956.39 9723.273   
005180 2022       2022-12-31       251038.0 32063.589498   
005600 2022       2022-12-31        61712.663909 73116.0   
...                                                  ...   
275535 2022       2022-03-31        4498.412 111.2256901   
       2023       2023-03-31       3819.836 79.558482923   
289724 2022       2022-12-31       1642.732 0.0869208615   
295786 2022       2022-12-31     129177.195 3.5451698548   
318474 2022       2022-12-31         4813.0 1649.4464065   

                                                      di_319415  
gvkey  fiscalyear periodenddate                                  
003413 2022       2022-12-31           1650402.398 2024652.4121  
003897 2022       2022-12-31           4025432.601 1005488.8192  
004094 2022       2022-12-31     197193.363 84877.747 39817.706  
005180 2022       2022-12-31           2436669.603 254265.62927  
005600 2022       2022-12-31            850262.94718 353150.506  
...                                                         ...  
275535 2022       2022-03-31              8751.123 2720.5732053  
       2023       2023-03-31              6836.914 1945.9953604  
289724 2022       2022-12-31              7071.309 0.3741598245  
295786 2022       2022-12-31           5009667.962 64.639735408  
318474 2022       2022-12-31            109611.983 3010.0308692  

[95 rows x 3 columns]

It is evident from the above that the gvkeys are mapping to multiple company ids in 
the same period and this is leading to multiple instances of the environment metrics

**Gvkey <-> Companyid Mapping**

- *gvkey -> company id*

In [188]:
raw_emissions_gvkey_companyid_1t1_conflicts = detect_1t1_conflicts(
    raw_emissions_df, 
    "gvkey", "companyid"
)

raw_emissions_gvkey_companyid_1t1_conflicts

,companyid
gvkey,
001004,NaN
001045,NaN
001050,NaN
001075,NaN
001076,NaN
...,...
362620,NaN
362683,NaN
362705,NaN


In [189]:
raw_emissions_gvkey_companyid_1t1_conflicts[
    pd.notnull(raw_emissions_gvkey_companyid_1t1_conflicts["companyid"])
]

,companyid
gvkey,
002856,259777 259777 3103613 3103613
003413,263295 6020983 6020983
003897,266598 1067744 1067744
004094,528325 27755 386000 386000
005180,275442 275442 5546439
...,...
275535,109366051 109366051 881331 881331
275839,53336883 5482350
289724,215005509 215005509 30941174


Notice that gvkey can map to multiple companyids, but remember that it 
maps to at least 1 company id because it backed by companyid.

- *company id -> gvkey*

In [190]:
raw_emissions_companyid_gvkey_1t1_conflicts = detect_1t1_conflicts(
    raw_emissions_df, 
    "companyid", "gvkey"
)

raw_emissions_companyid_gvkey_1t1_conflicts

,gvkey
companyid,
100000307,
100013,NaN
1000212,
1000277,
10004497,NaN
...,...
99996472,
99996476,
99996998,


In [193]:
raw_emissions_companyid_gvkey_1t1_conflicts[
    pd.notnull(raw_emissions_companyid_gvkey_1t1_conflicts["gvkey"]) 
    & (raw_emissions_companyid_gvkey_1t1_conflicts["gvkey"] == "")
]

,gvkey
companyid,
100000307,
1000212,
1000277,
10004521,
10005029,
...,...
99996472,
99996476,
99996998,


In [194]:
raw_emissions_companyid_gvkey_1t1_conflicts[
    pd.notnull(raw_emissions_companyid_gvkey_1t1_conflicts["gvkey"]) 
    & (raw_emissions_companyid_gvkey_1t1_conflicts["gvkey"] != "")
]

,gvkey
companyid,
104422749,037090 184982
106683684,285220 316585
1067744,003897 065089 003897 065089
1073371,100557 220942 100557 220942
112732,024616 145471 024616 145471
...,...
9683016,220688 247655
983017,007824 145270 007824 145270
9833116,242985 321595


Notice that companyid can both map to no or multiple gvkeys

**Institutionid <-> Companyid Mapping**

- *companyid -> institutionid*

In [195]:
raw_emissions_companyid_institutionid_1t1_conflicts = detect_1t1_conflicts(
    raw_emissions_df, 
    "companyid", "institutionid"
)

raw_emissions_companyid_institutionid_1t1_conflicts

,institutionid
companyid,
100000307,NaN
100013,NaN
1000212,NaN
1000277,NaN
10004497,NaN
...,...
99996472,NaN
99996476,NaN
99996998,NaN


In [196]:
raw_emissions_companyid_institutionid_1t1_conflicts[
    pd.notnull(raw_emissions_companyid_institutionid_1t1_conflicts["institutionid"])
]

,institutionid
companyid,


Hence, we can see companyid maps to exactly one institutionid.

- *institutionid -> companyid*

In [197]:
raw_emissions_institutionid_companyid_1t1_conflicts = detect_1t1_conflicts(
    raw_emissions_df, 
    "institutionid", "companyid"
)

raw_emissions_institutionid_companyid_1t1_conflicts

,companyid
institutionid,
11489,6520204 24951392 43964734 6520204 24951392 439...
11654,NaN
11679,8333444 24586834 8333444 24586834
11894,7925667 34873695 7925667 34873695
12062,NaN
...,...
118526590,NaN
118706040,NaN
118918440,NaN


In [198]:
raw_emissions_institutionid_companyid_1t1_conflicts[
    pd.notnull(raw_emissions_institutionid_companyid_1t1_conflicts["companyid"])
]

,companyid
institutionid,
11489,6520204 24951392 43964734 6520204 24951392 439...
11679,8333444 24586834 8333444 24586834
11894,7925667 34873695 7925667 34873695
13644,26465936 60478967 26465936 60478967
13959,246652 6167099 242354247 246652 6167099 242354247
...,...
109987465,
110365423,
111445983,


Here, we can see that while a company belongs to exactly one institution, an 
institution maps to many companys (none, one or potentially many).

**Gvkey <-> Institutionid Mapping**

- *gvkey -> institutionid*

In [201]:
raw_emissions_gvkey_institutionid_1t1_conflicts = detect_1t1_conflicts(
    raw_emissions_df, 
    "gvkey", "institutionid"
)

raw_emissions_gvkey_institutionid_1t1_conflicts

,institutionid
gvkey,
001004,NaN
001045,NaN
001050,NaN
001075,NaN
001076,NaN
...,...
362620,NaN
362683,NaN
362705,NaN


In [200]:
raw_emissions_gvkey_institutionid_1t1_conflicts[
    pd.notnull(raw_emissions_gvkey_institutionid_1t1_conflicts["institutionid"])
]

,institutionid
gvkey,
002856,4057039 4057039 4057076 4057076
003413,4057041 4057080 4057080
003897,4057044 4057083 4057083
004094,108462 4021861 4388004 4388004
005180,4806213 4806213 5053995
...,...
275535,4265945 4265945 4326804 4326804
275839,4295672 6343205
289724,4772912 4772912 6393031


Noting that there are no null values in institutionid, this means that gvkey 
corresponds to at least one, and potentially many, institutions.

- *institutionid -> gvkey*

In [203]:
raw_emissions_institutionid_gvkey_1t1_conflicts = detect_1t1_conflicts(
    raw_emissions_df, 
    "institutionid", "gvkey"
)

raw_emissions_institutionid_gvkey_1t1_conflicts

,gvkey
institutionid,
11489,
11654,
11679,
11894,
12062,
...,...
118526590,
118706040,
118918440,


In [206]:
raw_emissions_institutionid_gvkey_1t1_conflicts[
    pd.notnull(raw_emissions_institutionid_gvkey_1t1_conflicts["gvkey"]) 
    & (raw_emissions_institutionid_gvkey_1t1_conflicts["gvkey"] == "")
]

,gvkey
institutionid,
11489,
11654,
11679,
11894,
12062,
...,...
118526590,
118706040,
118918440,


In [207]:
raw_emissions_institutionid_gvkey_1t1_conflicts[
    pd.notnull(raw_emissions_institutionid_gvkey_1t1_conflicts["gvkey"]) 
    & (raw_emissions_institutionid_gvkey_1t1_conflicts["gvkey"] != "")
]

,gvkey
institutionid,
100165,002001 002002
100259,004708 027665 004708 027665
100391,008119 017095 008119 017095
101674,005849 039571
103042,012124 027867
...,...
6618361,204440 247501
6626040,278266 340246
6932029,100787 326859


We can see that institutionids map to many gvkeys (can be zero, one or multiple)

**Determine the number of gvkey identifiers that exist across all fiscal years and also the number that exist in each fiscal year**

First check whether `fiscalyear` is ever null

In [209]:
print("Is fiscalyear ever null:")
print('YES' if raw_emissions_df["fiscalyear"].isnull().any() else 'NO')

Is fiscalyear ever null:
NO


Check if there are any duplicate fiscal year entries for gvkeys

In [212]:
raw_emissions_gvkey_fiscalyear_dup = raw_emissions_df.groupby("gvkey").agg(
    {
        "fiscalyear": lambda vals: vals.duplicated().any(),
    }
)

raw_emissions_gvkey_fiscalyear_dup[
    raw_emissions_gvkey_fiscalyear_dup["fiscalyear"]
]

,fiscalyear
gvkey,
002856,True
003413,True
003897,True
004094,True
005180,True
...,...
271134,True
275535,True
289724,True


We can see there are, and this is due to the duplication from the varying 
`companyid`'s and `institutionid`'s

## Load Cid-Gvkey Mappings
> From the relevent CIQ Linking Web Query

In [233]:
cid_gvkey_df = pd.read_csv("cid_gvkey_map.csv")

cid_gvkey_df

,companyid,gvkey,startdate,enddate,companyname
0,18511,210835,B,E,3i Group plc
1,18527,210418,B,E,ABB Ltd
2,18671,29751,B,E,Albemarle Corporation
3,18711,28349,B,E,The Allstate Corporation
4,18749,64768,B,E,"Amazon.com, Inc."
...,...,...,...,...,...
24388,1849817514,361808,B,E,Kawan Renergy Berhad
24389,1855364409,358709,B,E,ELSA Solutions S.p.A.
24390,1855399529,358653,B,E,"SEIYU KOGYO Co.,Ltd."
24391,1859487646,359029,B,E,KET Inc.


Inspect the loaded datatypes

In [234]:
cid_gvkey_df.dtypes

companyid       int64
gvkey           int64
startdate      object
enddate        object
companyname    object
dtype: object

Filter the mappings to just those that are one-to-one, and keep note of these

In [235]:
cid_gvkey_1t1_df = cid_gvkey_df[(cid_gvkey_df["startdate"] == "B") & (cid_gvkey_df["enddate"] == "E")]

cid_gvkey_1t1_df

,companyid,gvkey,startdate,enddate,companyname
0,18511,210835,B,E,3i Group plc
1,18527,210418,B,E,ABB Ltd
2,18671,29751,B,E,Albemarle Corporation
3,18711,28349,B,E,The Allstate Corporation
4,18749,64768,B,E,"Amazon.com, Inc."
...,...,...,...,...,...
24388,1849817514,361808,B,E,Kawan Renergy Berhad
24389,1855364409,358709,B,E,ELSA Solutions S.p.A.
24390,1855399529,358653,B,E,"SEIYU KOGYO Co.,Ltd."
24391,1859487646,359029,B,E,KET Inc.


*Another approach for filtering mappings is considering where gvkey is not 
duplicated, as opposed to mappings that exist for all of time. This alternative 
approach may however lead to cases where the same company exists under 
different gvkeys (same companyid corresponds to multiple gvkeys). This was not 
chosen in our case.*

Verify that these mappings are indeed one-to-one

In [236]:
print(f"Gvkey duplicates: {cid_gvkey_1t1_df['gvkey'].duplicated().any()}")
print(f"Companyid duplicates: {cid_gvkey_1t1_df['companyid'].duplicated().any()}")

Gvkey duplicates: False
Companyid duplicates: False


The lack of gvkey duplicates indicates no redundancy and the lack of companyid 
duplicates indicates that the mappings are indeed one to one (i.e. they each go 
to unique companyids).

>Notice one-to-one mapping reduces the number of unique gvkeys from the ~24k to ~23k in this join table.

**Filter the main environment table using this join table, remembering the following facts:**
The raw main environment table has:
- ~1.8 million rows
- ~1.7 million unique companyid/institutionid values
- ~24k unique gvkey values

From the main environment table, filter entries to just those with gvkeys.

In [237]:
emissions_df = raw_emissions_df[pd.notnull(raw_emissions_df["gvkey"])]

Further filter to just the gvkeys that 1 to 1 map with companyid. This should 
leave one entry per fiscal year for each gvkey. 

*Note some gvkeys may not have an entry for certain fiscal years because that data is simply missing.*

In [238]:
emissions_df = emissions_df[
    np.in1d(emissions_df["gvkey"], cid_gvkey_1t1_df["gvkey"].astype(str))
]

emissions_df

,institutionid,fiscalyear,periodenddate,di_319413,di_319414,di_319415,companyid,gvkey,companyname,country
26,13959,2022,2022-12-31,18853.165084,72589.700681,1.323541e+06,246652,122594,State Farm Mutual Automobile Insurance Company,United States
29,13959,2023,2023-12-31,23250.617734,89521.063147,1.632254e+06,246652,122594,State Farm Mutual Automobile Insurance Company,United States
32,14193,2022,2022-12-31,16736.521444,51330.155513,7.097935e+05,417155,122554,United Services Automobile Association,United States
33,14193,2023,2023-12-31,19585.900250,60069.071644,8.306352e+05,417155,122554,United Services Automobile Association,United States
38,14467,2022,2022-12-31,799.396226,2451.712127,3.390228e+04,675974,263562,Everlake Life Insurance Company,United States
...,...,...,...,...,...,...,...,...,...,...
1796814,112329550,2022,2022-10-31,489.910950,1022.660743,6.379460e+03,1849817514,361808,Kawan Renergy Berhad,Malaysia
1803250,113511623,2022,2022-12-31,291.908186,183.331947,5.197957e+02,1855364409,358709,ELSA Solutions S.p.A.,Italy
1803251,113511665,2022,2022-09-30,12.539324,0.790235,6.400957e+00,1855399529,358653,"SEIYU KOGYO Co.,Ltd.",Japan
1803259,114230580,2022,2022-03-31,2.811856,0.177205,1.435370e+00,1859487646,359029,KET Inc.,Japan


Here, we check whether the above statement of exactly one entry per gvkey, 
fiscal year is true (barring the exception of missing data).

In [239]:
emissions_gvkey_fiscalyear_entries = emissions_df.groupby(["gvkey", "fiscalyear"]).size()

emissions_gvkey_fiscalyear_entries

gvkey   fiscalyear
100001  2022          1
        2023          1
100006  2022          1
100010  2022          1
100012  2022          1
                     ..
362683  2022          1
362705  2022          1
362758  2022          1
362761  2022          1
        2023          1
Length: 26407, dtype: int64

In [240]:
emissions_gvkey_fiscalyear_entries[
    emissions_gvkey_fiscalyear_entries != 1
]

Series([], dtype: int64)

The empty return for a number of entries different than 1 shows that 
we have indeed made gvkey, fiscal year unique.


In [227]:
emissions_df.nunique()["gvkey"]

19352

We can see that this process has reduced the number of unique `gvkey`'s to 
~19k now.

In [241]:
emissions_df

,institutionid,fiscalyear,periodenddate,di_319413,di_319414,di_319415,companyid,gvkey,companyname,country
26,13959,2022,2022-12-31,18853.165084,72589.700681,1.323541e+06,246652,122594,State Farm Mutual Automobile Insurance Company,United States
29,13959,2023,2023-12-31,23250.617734,89521.063147,1.632254e+06,246652,122594,State Farm Mutual Automobile Insurance Company,United States
32,14193,2022,2022-12-31,16736.521444,51330.155513,7.097935e+05,417155,122554,United Services Automobile Association,United States
33,14193,2023,2023-12-31,19585.900250,60069.071644,8.306352e+05,417155,122554,United Services Automobile Association,United States
38,14467,2022,2022-12-31,799.396226,2451.712127,3.390228e+04,675974,263562,Everlake Life Insurance Company,United States
...,...,...,...,...,...,...,...,...,...,...
1796814,112329550,2022,2022-10-31,489.910950,1022.660743,6.379460e+03,1849817514,361808,Kawan Renergy Berhad,Malaysia
1803250,113511623,2022,2022-12-31,291.908186,183.331947,5.197957e+02,1855364409,358709,ELSA Solutions S.p.A.,Italy
1803251,113511665,2022,2022-09-30,12.539324,0.790235,6.400957e+00,1855399529,358653,"SEIYU KOGYO Co.,Ltd.",Japan
1803259,114230580,2022,2022-03-31,2.811856,0.177205,1.435370e+00,1859487646,359029,KET Inc.,Japan


- todo: QC nans
- todo: write out the gvkeys

- todo: extract fundamentals data of the same fiscal years (also see the link between periodenddate and fiscal year)
- todo: market data of one year ahead from the fiscal/fundamentals data
  - or at least one month

## Load Market Data

In [23]:
market_df = pd.read_csv("phase1_market.csv")

market_df

,gvkeyx,prccm,datadate,conm,tic
0,150918,3379.0814,2022-01-31,S&P Global 1200 Index,I6UNK112
1,150918,3283.8722,2022-02-28,S&P Global 1200 Index,I6UNK112
2,150918,3361.5970,2022-03-31,S&P Global 1200 Index,I6UNK112
3,150918,3089.8620,2022-04-30,S&P Global 1200 Index,I6UNK112
4,150918,3097.8104,2022-05-31,S&P Global 1200 Index,I6UNK112
5,150918,2827.3566,2022-06-30,S&P Global 1200 Index,I6UNK112
6,150918,3032.1650,2022-07-31,S&P Global 1200 Index,I6UNK112
7,150918,2899.3630,2022-08-31,S&P Global 1200 Index,I6UNK112
8,150918,2619.9194,2022-09-30,S&P Global 1200 Index,I6UNK112
9,150918,2795.4613,2022-10-31,S&P Global 1200 Index,I6UNK112


## Missing data checks to prepare for imputation

In [ ]:
""" Missing Data Checks """

In [4]:
""" Number of unique companies across all the years """
np.unique(df["gvkey"]).size # roughly 30k

31511

In [10]:
""" Now determine the number of unique companies in the 
different years
"""
print(f"All years: {np.unique(df['fiscalyear'])}")

for year in np.unique(df['fiscalyear']):
    print(f"{year} => {np.unique(df[df['fiscalyear'] == year]['gvkey']).size}")

All years: [2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015
 2016 2017 2018 2019 2020 2021 2022 2023]
2002 => 1763
2003 => 2000
2004 => 2885
2005 => 3880
2006 => 4170
2007 => 4307
2008 => 4269
2009 => 4563
2010 => 4723
2011 => 4833
2012 => 4868
2013 => 5757
2014 => 6154
2015 => 6235
2016 => 13882
2017 => 14786
2018 => 16979
2019 => 17378
2020 => 23353
2021 => 23386
2022 => 24053
2023 => 9725


In [ ]:
# print(df[])

## Write out all the unique gvkeys

## Read in the daily security prices csv

In [16]:
returns_df = pd.read_csv("daily.csv")

/var/folders/y5/ty9zs0px6gbg3f5k7p4jqh4m0000gn/T/ipykernel_80834/789980338.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  returns_df = pd.read_csv("daily.csv")


In [17]:
returns_df

,gvkey,iid,datadate,conm,trfd,isin,county,sic
0,1166,01W,2002-01-02,ASM INTERNATIONAL NV,1.000000,NL0000334118,NaN,3559.0
1,1166,01W,2002-01-03,ASM INTERNATIONAL NV,1.000000,NL0000334118,NaN,3559.0
2,1166,01W,2002-01-04,ASM INTERNATIONAL NV,1.000000,NL0000334118,NaN,3559.0
3,1166,01W,2002-01-07,ASM INTERNATIONAL NV,1.000000,NL0000334118,NaN,3559.0
4,1166,01W,2002-01-08,ASM INTERNATIONAL NV,1.000000,NL0000334118,NaN,3559.0
...,...,...,...,...,...,...,...,...
113958618,362779,01W,2024-08-06,NOVAMARINE SPA,1.145429,IT0005605701,NaN,NaN
113958619,362779,01W,2024-08-07,NOVAMARINE SPA,1.145429,IT0005605701,NaN,NaN
113958620,362779,01W,2024-08-08,NOVAMARINE SPA,1.145429,IT0005605701,NaN,NaN
113958621,362779,01W,2024-08-09,NOVAMARINE SPA,1.145429,IT0005605701,NaN,NaN


## Check the intersect between the gvkeys of the carbon emissions dataset and the daily prices dataset

In [31]:
# unique non-nan in environment
env_gvkeys = np.unique(df[~np.isnan(df["gvkey"])]["gvkey"])

print(env_gvkeys.size)

31510


In [32]:
# unique non-nan in returns
returns_gvkeys = np.unique(returns_df[~np.isnan(returns_df["gvkey"])]["gvkey"])

print(returns_gvkeys.size)

24110


In [33]:
env_ret_common_gvkeys = np.intersect1d(
    env_gvkeys, 
    returns_gvkeys, 
    assume_unique=True
)

print(env_ret_common_gvkeys.size)

24110


In [34]:
""" Check the missing gvkeys to see what country they are from """
missing_gvkeys = np.setdiff1d(env_gvkeys, env_ret_common_gvkeys, assume_unique=True)

print(missing_gvkeys.size)

7400


In [77]:
""" Create representatives and break down their distribution """
df_missing_reprs_idx = [(df["gvkey"] == gvkey).idxmax() for gvkey in missing_gvkeys]
missing_dist = df.iloc[df_missing_reprs_idx][["gvkey", "country"]].groupby(
    "country"
).count().reset_index().sort_values('gvkey', ascending=False)

In [78]:
missing_dist

,country,gvkey
79,United States,5220
10,Canada,803
78,United Kingdom,271
13,China,113
24,France,111
...,...,...
58,Peru,1
57,Panama,1
56,Pakistan,1
48,Marshall Islands,1


In [79]:
missing_dist[missing_dist["country"] != "United States"]["gvkey"].sum()

2180

## Checks on Duplicates

In [15]:
company_dups = df[df.duplicated('gvkey', keep=False) == True].sort_values(by="gvkey")

In [26]:
last_iid = None
last_fyears = set()
for i, row in company_dups.iterrows():
    current_iid = row["institutionid"]
    current_fyear = row["fiscalyear"]
    if last_iid is not None and current_iid == last_iid:
        if current_fyear in last_fyears:
            raise Exception(f"Fyear clash!, iid: {current_iid}, prev_fyears: {last_fyears}, clash: {current_fyear}")
        else:
            last_fyears.add(current_fyear)
    else:
        last_iid = current_iid
        last_fyears.clear()
        last_fyears.add(current_fyear)

print("No issues...")

Exception: Fyear clash!, iid: 4415462, prev_fyears: {2022}, clash: 2022

In [31]:
company_dups.head(20)

,periodid,institutionid,reportedcurrencyisocode,tcprimarysectorid,fiscalyear,periodenddate,di_319380,di_319381,di_319382,di_319383,...,streetaddress3,streetaddress4,zipcode,yearfounded,monthfounded,dayfounded,officephonevalue,otherphonevalue,officefaxvalue,webpage
5,30D218CF-2E2A-46B4-AF72-CADF593290E8,4074603,USD,713A00,2022,01/01/2023,0.285,0.021,2.079,0.153,...,NaN,NaN,76011,1961.0,NaN,NaN,972 595 5000,NaN,NaN,www.sixflags.com
2503,1989BCA5-218F-4857-A3CE-1F2114D67F8E,4074603,USD,713A00,2023,31/12/2023,0.302,0.021,2.098,0.147,...,NaN,NaN,76011,1961.0,NaN,NaN,972 595 5000,NaN,NaN,www.sixflags.com
4988,5B249039-D1E2-43D2-825B-DDD233FE2FE0,4996548,USD,561300,2023,31/12/2023,0.015,0.005,0.125,0.040,...,NaN,NaN,75024,2007.0,NaN,NaN,972 692 2400,NaN,NaN,bgsf.com
40,5F205052-5493-41C8-9C8C-1E0FE1FC5DC0,4996548,USD,561300,2022,01/01/2023,0.014,0.005,0.122,0.041,...,NaN,NaN,75024,2007.0,NaN,NaN,972 692 2400,NaN,NaN,bgsf.com
1,01C53196-7DD9-42C7-9D3B-F152BFB3A364,4054841,USD,445000A,2022,01/01/2023,3.061,0.003,186.279,0.203,...,NaN,NaN,1506 MA,1867.0,NaN,NaN,31 88 659 9111,NaN,NaN,www.aholddelhaize.com
2462,793C84DE-3E9D-4CE0-8768-647BA85F9272,4054841,USD,445000A,2023,31/12/2023,3.183,0.003,181.084,0.189,...,NaN,NaN,1506 MA,1867.0,NaN,NaN,31 88 659 9111,NaN,NaN,www.aholddelhaize.com
110,A706AF20-7252-4C95-AA58-2481363942C5,10175068,USD,722000,2022,02/01/2023,0.095,0.053,0.696,0.389,...,NaN,NaN,33309,2011.0,NaN,NaN,954-618-2000,NaN,NaN,www.burgerfi.com
6071,96731D0E-B576-41E4-8386-DAD27811DF60,10175068,USD,722000,2023,01/01/2024,0.092,0.054,0.625,0.368,...,NaN,NaN,33309,2011.0,NaN,NaN,954-618-2000,NaN,NaN,www.burgerfi.com
99,0446986D-7007-4B72-AA5B-49AA5ADB4CF2,28295169,USD,541512,2022,01/01/2023,0.042,0.020,0.362,0.170,...,NaN,NaN,55425,2016.0,NaN,NaN,952 851 5200,NaN,NaN,www.skywatertechnology.com
6011,9AABC54A-2E82-4C6A-99E2-489EF08AE342,28295169,USD,541512,2023,31/12/2023,0.051,0.018,0.420,0.147,...,NaN,NaN,55425,2016.0,NaN,NaN,952 851 5200,NaN,NaN,www.skywatertechnology.com


In [32]:
company_dups[company_dups["institutionid"] == 4415462]

,periodid,institutionid,reportedcurrencyisocode,tcprimarysectorid,fiscalyear,periodenddate,di_319380,di_319381,di_319382,di_319383,...,streetaddress3,streetaddress4,zipcode,yearfounded,monthfounded,dayfounded,officephonevalue,otherphonevalue,officefaxvalue,webpage
23,5282F3CE-9CFA-4830-9116-3B36EFDE5089,4415462,USD,52A000,2022,01/01/2023,0.001,0.001,0.054,0.037,...,NaN,NaN,050059,1993.0,NaN,NaN,7 727 244 5484,NaN,7 727 244 5480,www.homecredit.kz
24,5282F3CE-9CFA-4830-9116-3B36EFDE5089,4415462,USD,52A000,2022,01/01/2023,0.001,0.001,0.054,0.037,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


array(['United States', 'Netherlands', 'United Kingdom', 'Canada',
       'Kazakhstan', 'Belarus', 'Australia', 'Belgium', 'Austria',
       'Finland', 'Ireland', nan, 'Singapore', 'France', 'Denmark',
       'Japan', 'Israel', 'Italy', 'South Africa', 'Thailand', 'Germany',
       'China', 'Hong Kong', 'Luxembourg', 'India', 'Switzerland',
       'Malaysia', 'South Korea', 'Kenya', 'New Zealand', 'Spain',
       'Pakistan', 'Saudi Arabia', 'Sweden', 'British Virgin Islands',
       'Kuwait', 'Turkey', 'Philippines', 'Mauritius', 'Bangladesh',
       'Cayman Islands', 'Botswana', 'Egypt', 'Malta', 'Malawi',
       'Jamaica', 'Bermuda', 'Colombia', 'Mexico', 'Norway', 'Brazil',
       'Bahrain', 'Morocco', 'Indonesia', 'Romania', 'Russia',
       'Ivory Coast', 'Tunisia', 'Greece', 'Vietnam', 'Taiwan', 'Nigeria',
       'Oman', 'Qatar', 'Portugal', 'United Arab Emirates', 'Jersey',
       'Poland', 'Bulgaria', 'Chile', 'Reunion', 'Ghana', 'Monaco',
       'Bahamas', 'Guernsey'], dtype=o